# calculate the RM for each stress-testing scenario

This notebook calucates the reserve margin for all scenarios


In [3]:
import os
import pandas as pd
from pathlib import Path
import logging


In [2]:
# Load the CSV file into a DataFrame
linkages = pd.read_csv('/Users/liyangwang/kitxDMDU_analysis/2045-Core_test/filtered_linkages.csv')

# Filter the DataFrame
filteredLinkages = linkages[(linkages['linkage'] == 'ResourceToZone') & (linkages['component_to'] == 'CAISO')]

filteredLinkages

,component_from,linkage,scenario,component_to
0,CAISO_Hydro,ResourceToZone,Base,CAISO
1,CAISO_Shed_DR_Existing,ResourceToZone,Shed DR LSE Plans: No LSE Plans,CAISO
2,CAISO_Shed_DR_Existing,ResourceToZone,Shed DR LSE Plans: 25 MMT,CAISO
3,CAISO_Shed_DR_Existing,ResourceToZone,Shed DR LSE Plans: 30 MMT,CAISO
4,Shed_ag_Pumping,ResourceToZone,Base,CAISO
...,...,...,...,...
186,Flex_LDV_V2G_Com,ResourceToZone,Mid_LDV,CAISO
187,Flex_LDV_V1G_Res,ResourceToZone,High_LDV,CAISO
188,Flex_LDV_V1G_Com,ResourceToZone,High_LDV,CAISO
189,Flex_LDV_V2G_Res,ResourceToZone,High_LDV,CAISO


In [4]:
import logging

def setup_logging(log_file_path):
    logging.basicConfig(filename=log_file_path, level=logging.INFO, 
                        format='%(asctime)s - %(levelname)s - %(message)s')

def process_rm(policy_folder, output_file_path, log_file_path):
    setup_logging(log_file_path)
    reserveMarginList = []

    for root, dirs, files in os.walk(policy_folder):
        if 'expressions' in dirs:
            try:
                expressions_dir = os.path.join(root, 'expressions')
                parameters_dir = os.path.join(root, 'parameters')
                rep_periods_dir = os.path.join(root, 'temporal_settings')
                variables_dir = os.path.join(root, 'variables')
                temporal_settings_dir = os.path.join(root, 'temporal_settings')
                
                plant_capacity_file = os.path.join(expressions_dir, 'Plant_Provide_Power_Capacity_In_Timepoint_MW.csv')
                input_load_file = os.path.join(parameters_dir, 'input_load_mw.csv')
                soc_intra_period_file = os.path.join(variables_dir, 'SOC_Intra_Period.csv')
                provide_power_mw_file = os.path.join(variables_dir, 'Provide_Power_MW.csv')
                rep_periods_file = os.path.join(temporal_settings_dir, 'rep_periods.csv')
                transmit_power_mw_file = os.path.join(variables_dir, 'Transmit_Power_MW.csv')

                plantCapacity = pd.read_csv(plant_capacity_file)
                inputLoad = pd.read_csv(input_load_file)
                socIntra = pd.read_csv(soc_intra_period_file)
                providePower = pd.read_csv(provide_power_mw_file)
                repPeriod = pd.read_csv(rep_periods_file)
                transmitPower = pd.read_csv(transmit_power_mw_file)

                logging.info(f"Loaded CSV files from {root}")

                filteredPlantCapacity = plantCapacity[plantCapacity['PLANTS_THAT_PROVIDE_POWER'].isin(filteredLinkages['component_from'])]
                resources_with_storage = socIntra['RESOURCES_WITH_STORAGE'].unique()

                truePlantCapacity = pd.merge(
                    filteredPlantCapacity,
                    providePower,
                    on=['MODEL_YEARS', 'REP_PERIODS', 'HOURS', 'PLANTS_THAT_PROVIDE_POWER'],
                    how='inner'
                )
                mask = truePlantCapacity['PLANTS_THAT_PROVIDE_POWER'].isin(resources_with_storage)
                truePlantCapacity.loc[mask, 'Plant_Provide_Power_Capacity_In_Timepoint_MW'] = truePlantCapacity.loc[mask, 'Provide_Power_MW']

                try:
                    truePlantCapacity = pd.merge(truePlantCapacity, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
                    truePlantCapacity['datetime'] = pd.to_datetime(truePlantCapacity['MODEL_YEARS'].astype(str) + '-' + truePlantCapacity['0'].str[5:], errors='coerce')
                    truePlantCapacity['datetime'] = truePlantCapacity['datetime'] + pd.to_timedelta(truePlantCapacity['HOURS'], unit='h')
                    truePlantCapacity.drop(columns=['period', '0'], inplace=True)
                    logging.info("Converted MODEL_YEARS, REP_PERIODS, and HOURS into datetime successfully.")
                except Exception as e:
                    logging.error(f"Error converting MODEL_YEARS, REP_PERIODS, and HOURS into datetime: {e}")

                try:
                    caisoLoad = inputLoad[inputLoad['ZONES'] == 'CAISO'].copy()
                    caisoLoad[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']] = caisoLoad[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']].astype(int)
                    caisoLoad = pd.merge(caisoLoad, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
                    caisoLoad['datetime'] = pd.to_datetime(caisoLoad['MODEL_YEARS'].astype(str) + '-' + caisoLoad['0'].str[5:], errors='coerce') + pd.to_timedelta(caisoLoad['HOURS'], unit='h')
                    caisoLoad = caisoLoad.sort_values(by='datetime')
                    logging.info("Calculated hourly input load successfully.")
                except Exception as e:
                    logging.error(f"Error calculating hourly input load: {e}")
                    
                try:
                    caisoImportExport = transmitPower[transmitPower['TRANSMISSION_LINES'].str.contains('to CAISO')].copy()
                    caisoImportExport[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']] = caisoImportExport[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']].astype(int)
                    caisoImportExport = pd.merge(caisoImportExport, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
                    caisoImportExport['datetime'] = pd.to_datetime(caisoImportExport['MODEL_YEARS'].astype(str) + '-' + caisoImportExport['0'].str[5:], errors='coerce') + pd.to_timedelta(caisoImportExport['HOURS'], unit='h')
                    caisoImportExport = caisoImportExport.sort_values(by='datetime')
                    caisoImportExport = caisoImportExport.groupby('datetime')['Transmit_Power_MW'].sum().reset_index()
                    logging.info("Calculated hourly import/export successfully.")
                except Exception as e:
                    logging.error(f"Error calculating import/export : {e}")

                try:
                    reserveMargin = truePlantCapacity.groupby('datetime')['Plant_Provide_Power_Capacity_In_Timepoint_MW'].sum().reset_index()
                    reserveMargin = reserveMargin.merge(caisoImportExport, on='datetime')
                    reserveMargin = reserveMargin.merge(caisoLoad, on='datetime')
                    reserveMargin['PRM'] = ((reserveMargin['Plant_Provide_Power_Capacity_In_Timepoint_MW'] + reserveMargin['Transmit_Power_MW']  - reserveMargin['input_load_mw']) / reserveMargin['input_load_mw']) * 100
                    logging.info("Calculated reserve margin successfully.")
                except Exception as e:
                    logging.error(f"Error calculating reserve margin: {e}")

                parts = root.split('/')
                part1 = parts[-3].replace('2010-2020ML', '-')
                part2 = parts[-2].replace('2045-', '')
                case_name = f"{part1}{part2}"
                reserveMargin['case'] = case_name
                reserveMarginList.append(reserveMargin)
            except FileNotFoundError as e:
                logging.error(f"File not found: {e}")
                continue

    RMall_df = pd.concat(reserveMarginList, ignore_index=True)
    RMall_df.drop(columns=['ZONES', 'MODEL_YEARS', 'REP_PERIODS', 'HOURS', 'period', '0'], inplace=True)
    RMall_df.to_csv(output_file_path, index=False)
    logging.info(f"Saved RMall_df to {log_file_path}")
    return RMall_df

In [ ]:
#inputs for the function process_rm
Core = '/Users/liyangwang/kitxDMDU_analysis/2045-Core_test'  # Replace with your root directory
outputCoreRM = '/Users/liyangwang/kitxDMDU_analysis/outputs/2045-Core_test_PRM.csv'
CoreLog = '/Users/liyangwang/kitxDMDU_analysis/coretest.log'

#run function below
CorePRM = process_rm(Core, outputCoreRM ,CoreLog)


CorePRM

,datetime,Plant_Provide_Power_Capacity_In_Timepoint_MW,Transmit_Power_MW,input_load_mw,PRM,case
0,2045-01-01 00:00:00,43355.406,5613.877,45428.334,7.794583,Baseline2_45_CESM2_2015
1,2045-01-01 01:00:00,40984.366,6415.875,44053.125,7.597908,Baseline2_45_CESM2_2015
2,2045-01-01 02:00:00,36493.329,6298.566,39103.021,9.433731,Baseline2_45_CESM2_2015
3,2045-01-01 03:00:00,34172.467,5328.104,34990.282,12.890119,Baseline2_45_CESM2_2015
4,2045-01-01 04:00:00,32244.839,4984.941,32568.934,14.310711,Baseline2_45_CESM2_2015
...,...,...,...,...,...,...
26275,2045-12-31 19:00:00,29832.350,7891.519,31868.455,18.373699,Baseline2_45_CESM2_2016
26276,2045-12-31 20:00:00,29832.350,8003.931,32701.145,15.703230,Baseline2_45_CESM2_2016
26277,2045-12-31 21:00:00,29832.350,9341.065,35129.153,11.512552,Baseline2_45_CESM2_2016
26278,2045-12-31 22:00:00,29832.350,10715.197,36854.356,10.021043,Baseline2_45_CESM2_2016
